# Week 8 Coding Homework 3
HW 3 - Use Keras Sequential APIs to develop a 2-layer neural network to solve the cat vs. non-cat classification problem (from Week 3 - HW2)

In [1]:
import time
import numpy as np
import h5py
import scipy
from PIL import Image
from scipy import ndimage
from dnn_app_utils import *
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import classification_report, confusion_matrix

%load_ext autoreload
%autoreload 2

np.random.seed(1)
tf.keras.utils.set_random_seed(1234)

In [2]:
train_x_orig, train_y, test_x_orig, test_y, classes = load_data()

In [3]:
# Prepare data
m_train = train_x_orig.shape[0]
num_px = train_x_orig.shape[1]
m_test = test_x_orig.shape[0]
train_x_flatten = train_x_orig.reshape(train_x_orig.shape[0], -1).T
test_x_flatten = test_x_orig.reshape(test_x_orig.shape[0], -1).T

# Standardize data to have feature values between 0 and 1.
train_x = train_x_flatten / 255.
test_x = test_x_flatten / 255.

In [4]:
# Declare the model structure
model = keras.Sequential()
model.add(keras.layers.InputLayer(shape=(train_x.shape[0],)))
model.add(keras.layers.Dense(10, activation='elu', kernel_initializer='he_normal'))
model.add(keras.layers.Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 10)             │       122,890 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 122,901 (480.08 KB)

 Trainable params: 122,901 (480.08 KB)

 Non-trainable params: 0 (0.00 B)

In [5]:
model.compile(optimizer='nadam', 
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [6]:
history = model.fit(train_x.T, train_y.T, epochs=150, batch_size=32)

Epoch 1/150
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6555 - loss: 1.2550  
Epoch 2/150
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6555 - loss: 0.6532 
Epoch 3/150
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6555 - loss: 0.6516 
Epoch 4/150
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6555 - loss: 0.6504 
Epoch 5/150
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6555 - loss: 0.6513 
Epoch 6/150
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6555 - loss: 0.6477 
Epoch 7/150
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6555 - loss: 0.6474 
Epoch 8/150
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6555 - loss: 0.6466 
Epoch 9/150
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6555 - loss: 0.6458 
Epoch 10/150
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6555 - loss: 0.6416 
Epoch 11/150
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6651 - loss: 0.6316 
Epoch 12/150
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6746 - lo

In [7]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_x.T, test_y.T, verbose=1)
print(f"Test Accuracy: {test_accuracy:.4f}")

# Make predictions on the test set
predictions = model.predict(test_x.T)
predicted_classes = (predictions > 0.5).astype(int)

# Calculate additional metrics
print("\nClassification Report:")
print(classification_report(test_y.T, predicted_classes, target_names=['Non-Cat', 'Cat']))

print("\nConfusion Matrix:")
print(confusion_matrix(test_y.T, predicted_classes))

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.7800 - loss: 1.2179
Test Accuracy: 0.7800
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step

Classification Report:
              precision    recall  f1-score   support

     Non-Cat       0.65      0.76      0.70        17
         Cat       0.87      0.79      0.83        33

    accuracy                           0.78        50
   macro avg       0.76      0.78      0.76        50
weighted avg       0.79      0.78      0.78        50


Confusion Matrix:
[[13  4]
 [ 7 26]]


Using some additional tricks (He initialization, ELU activation, and Nadam optimizer) we were able to get 78% accuracy on the test set vs 72% in the previous homework. But it took some experimentation to find settings that worked. Some models were much worse than 72%.